In [9]:
import numpy as np
import math
import os
import gdal
import xarray as xr
from skimage.measure import block_reduce
from ftplib import FTP
import subprocess
import rioxarray
import time as T
import datetime

class SGLI_L2_Downloader:
    
    def __init__ (self, User_ID , ProductName, Ver, Date , Tile , Obrit ,ParaVer, Target_path):
        self.User_ID = User_ID
        self.ProductName = ProductName
        self.Ver = Ver
        self.Date = Date
        self.Tile = Tile
        self.Obrit = Obrit
        self.ParaVer = ParaVer
        self.Target_path = Target_path
        
        
    def path (self):
        return '/standard/GCOM-C/GCOM-C.SGLI/L2.LAND.{}/{}/{}/{}/{}'.format(self.ProductName, self.Ver, self.Date[0:4], self.Date[4:6], self.Date[6:8])

    def filename (self):
        return 'GC1SG1_{}{}01D_T{}_L2SG_{}Q_{}00{}.h5'.format(self.Date, self.Obrit, self.Tile, self.ProductName, self.Ver, self.ParaVer)
    
    def login (self):
        ftp_addr = 'ftp.gportal.jaxa.jp'
        f = FTP(ftp_addr)
        f.login(self.User_ID,'anonymous')
        
        return f
    
    def download (self, f):
        remote_filepath = self.path()
        f.cwd(remote_filepath)
        list=f.nlst()

        bufsize=1024
        data = open ('{}{}'.format(self.Target_path, self.filename()), 'wb')
        filename = 'RETR ' + self.filename()
        f.retrbinary(filename,data.write,bufsize)
        
    def logout (self, f):
        f.quit()
    
def reporjection_GEO(filename,Band,Target_path):
    
    #print(path)
    subprocess.check_output('/media/liwei/Data/GCOM-C_RSRF/SGLI_geo_map_linux.exe {} -d Geometry_data/{} -o {} -s 7.5'.format(filename,Band,Target_path[:-1]), shell=True) 
    
def mkdir(path):
    folder = os.path.exists(path) 
    if not folder:
        os.makedirs(path)
        
    
def SGLI_2_AHI_GEO(filepath):
    geotiff_da = rioxarray.open_rasterio(filepath)
    data = geotiff_da.interp(x=lon_y,y=lat_x,method="nearest")
    data = np.where(data != -32768, data ,np.nan)
    data = data * 0.01
    data = data.reshape(40,40)
    data = block_reduce(data, block_size=(2,2), func=np.nanmean)
    return data

def month_size(month,YYYY):
    big_month=['01','03','05','07','08','10','12']
    small_month=['04','06','09','11']
    if month in big_month:
        s = 31
    elif month in small_month:
        s = 30
    elif month == '02':
        if (int(YYYY) % 4 == 0 and int(YYYY) % 100 != 0) or int(YYYY) % 400 == 0:
            s = 29
        else:
            s = 28
    return s

def get_site_range(lat,lon):
    crow = int(60 - lat) / 0.04
    ccol = int(lon - 85) / 0.04
    return int(crow-2),int(crow+2),int(ccol-2),int(ccol+2)

def download_tile(date,save_tile_path,tile):
    g = SGLI_L2_Downloader(
        'galiwei ' ,
        'RSRF', 
        '2', 
         date , 
         tile, 
        'D' , 
        '0' ,
         save_tile_path
        )
    f = g.login()
    if os.path.exists(save_tile_path + g.filename()):
        return g.filename()
    
    else :
        g.download(f)
        g.logout(f)
        return g.filename()
    
def download_AHI(target_path , date, time, angle):
    
    date1 = date[0:6]
    date2 = date
    AHI_date = date2 + time
    sza_file_name = '{}.sun.zth.fld.4km.bin.bz2'.format(AHI_date)
    saa_file_name = '{}.sun.azm.fld.4km.bin.bz2'.format(AHI_date)
    
    
    if angle == 'sza':
        if os.path.exists(target_path + sza_file_name[:-4]):
            return sza_file_name[:-4]
    
        else:
            ftp = FTP()
            ftp.connect('hmwr829gr.cr.chiba-u.ac.jp', 21)
            ftp.login()
            path = '/gridded/FD/V20190123/{}/4KM/{}/'.format(date1,date2)
            ftp.cwd(path)
            remote_list=ftp.nlst()
            bufsize=1024*1024
            if sza_file_name in remote_list:
                data = open(target_path + sza_file_name,'wb')
                filename='RETR '+ sza_file_name
                ftp.retrbinary(filename,data.write,bufsize)
                ftp.quit()

                return sza_file_name[:-4]
            else:
#                 print('AHI NO DATA')
                return None
        
    elif angle == 'saa':
        if os.path.exists(target_path + saa_file_name[:-4]):
            return saa_file_name[:-4]
    
        else:
            ftp = FTP()
            ftp.connect('hmwr829gr.cr.chiba-u.ac.jp', 21)
            ftp.login()
            path = '/gridded/FD/V20190123/{}/4KM/{}/'.format(date1,date2)
            ftp.cwd(path)
            remote_list=ftp.nlst()
            bufsize=1024*1024
            if saa_file_name in remote_list:
                data = open(target_path + saa_file_name,'wb')
                filename='RETR '+ saa_file_name
                ftp.retrbinary(filename,data.write,bufsize)
                ftp.quit()
                return saa_file_name[:-4]
            else:
#                 print('AHI NO DATA')
                return None
    
def Decompression(t,fn):
    if os.path.exists('{}.bz2'.format(t + fn)):
        p = subprocess.Popen('lbzip2 -d {}.bz2'.format(t + fn),shell=True)
        p.communicate()

In [10]:
YYYY='2019'
# MM = ['12']

# DD = ['21','22','23','24','25','26','27','28']
MM = ['01','02','03','04','05','06','07','08','09','10','11','12']
DD=['01','02','03','04','05','06','07','08','09','10',\
    '11','12','13','14','15','16','17','18','19','20',\
    '21','22','23','24','25','26','27','28','29','30',\
    '31']
HH=['09','10','11','12','13','14','15','16','17']
MIN=['00','10','20','30','40','50']

band = ['Solar_azimuth_PL', 'Solar_zenith_PL','Sensor_zenith_PL', 'Sensor_azimuth_PL']

jet = 5

site_name = 'India'

# site = [-41.5,172.5]
# site = [-44.5,170.5]
# tile = '1330'

site = [27.5,97]
tile = '0626'


MATCH_RESULT = []
#Save path
reporjection_file = '/media/liwei/Data/GCOM-C_RSRF_Valid/reporjection/'
tile_file = '/media/liwei/Data/GCOM-C_RSRF_Valid/tile/'
AHI_SAA_t = '/media/liwei/Data/AHI_Angle/Solar_azimuth_angle/'
AHI_SZA_t = '/media/liwei/Data/AHI_Angle/Solar_zenith_angle/'



urow,drow,lcol,rcol = get_site_range(site[0],site[1])
lat_x=np.linspace(site[0]+0.05,site[0]-0.0475,40)
lon_y=np.linspace(site[1]-0.05,site[1]+0.0475,40)


# AHI VZA/VAA Load
with open('/media/liwei/Data/AHI_Angle/AHI_VZA.dat','rb') as fp:
    AHI_VZA = np.frombuffer(fp.read()).reshape(3000,3000)[urow:drow,lcol:rcol]
AHI_VZA = np.nanmean(AHI_VZA)
with open('/media/liwei/Data/AHI_Angle/AHI_VAA.dat','rb') as fp:
    AHI_VAA = np.frombuffer(fp.read()).reshape(3000,3000)[urow:drow,lcol:rcol]
AHI_VAA = np.nanmean(AHI_VAA)

In [11]:
for m in range(len(MM)):
    s = month_size(MM[m],YYYY)
    for d in range(s):
        date = YYYY+MM[m]+DD[d]
        
#         SGLI file name
        SGLI_VZA_filename = 'GC1SG1_{}D01D_T{}_L2SG_RSRFQ_2000_Sensor_zenith_PL.tif'.format(date,tile)
        SGLI_VAA_filename = 'GC1SG1_{}D01D_T{}_L2SG_RSRFQ_2000_Sensor_azimuth_PL.tif'.format(date,tile)
        SGLI_SZA_filename = 'GC1SG1_{}D01D_T{}_L2SG_RSRFQ_2000_Solar_zenith_PL.tif'.format(date,tile)
        SGLI_SAA_filename = 'GC1SG1_{}D01D_T{}_L2SG_RSRFQ_2000_Solar_azimuth_PL.tif'.format(date,tile)
        
        
#         Reporjection file exist
        if not os.path.exists(reporjection_file + SGLI_VZA_filename):
            
#             print('Start SGLI Download and Reporjection.')
            fn = download_tile(date,tile_file,tile)
            
            for b in band:
                reporjection_GEO( tile_file+fn , b , reporjection_file)
#             print('Reporjection Finsihed!')
             
        if os.path.exists(reporjection_file + SGLI_VZA_filename):       
            SGLI_VZA_PL = SGLI_2_AHI_GEO(reporjection_file + SGLI_VZA_filename)
            SGLI_VZA_PL = np.nanmean(SGLI_VZA_PL)


            VZA_Matching_PL = abs(math.cos(math.radians(SGLI_VZA_PL)) - math.cos(math.radians(AHI_VZA)))
    #         print("AHI_VZA:",AHI_VZA,"SGLI_VZA:",SGLI_VZA_PL)
    #         print(VZA_Matching_PL)


            if VZA_Matching_PL < 0.02:
    #             print("AHI_VZA:",AHI_VZA,"SGLI_VZA:",SGLI_VZA_PL)
    #             print(VZA_Matching_PL)

    #             For loop for min
                for h in HH:
                    for minu in MIN:
                        date_old = datetime.datetime.strptime('{}-{}-{} {}:{}'.format(YYYY,MM[m],DD[d],h,minu), '%Y-%m-%d %H:%M')
                        delta = datetime.timedelta(hours = jet)
                        d_new = date_old - delta

                        date_new = d_new.strftime('%Y') + d_new.strftime('%m') + d_new.strftime('%d')
                        TIME = d_new.strftime('%H') + d_new.strftime('%M')


    #                     Download CEReS AHI Solar angle 
                        AHI_SAA_file_name = download_AHI(AHI_SAA_t , date_new, TIME, 'saa')
                        if AHI_SAA_file_name != None:

                            Decompression(AHI_SAA_t , AHI_SAA_file_name)
        #                     AHI SAA Load
                            with open(AHI_SAA_t + AHI_SAA_file_name, 'rb') as fp:
                                AHI_SAA = np.frombuffer(fp.read(), dtype='>f4').reshape(3000,3000)[urow:drow,lcol:rcol]
                            AHI_SAA = np.mean(AHI_SAA)

        #                     SGLI SAA/VAA Load
                            SGLI_VAA_PL = np.nanmean(SGLI_2_AHI_GEO(reporjection_file + SGLI_VAA_filename))
                            SGLI_SAA_PL = np.nanmean(SGLI_2_AHI_GEO(reporjection_file + SGLI_SAA_filename))

        #                     print(SGLI_VAA_PL,SGLI_SAA_PL)

        #                     RAA Calulation
                            SGLI_RAA_PL = abs(SGLI_SAA_PL - SGLI_VAA_PL)
                            if SGLI_RAA_PL > 180:
                                SGLI_RAA_PL = 360 - SGLI_RAA_PL

                            AHI_RAA = abs(AHI_SAA - AHI_VAA)
                            if AHI_RAA > 180:
                                AHI_RAA = 360 - AHI_RAA                        
        #                     print(AHI_RAA,SGLI_RAA_PL)



                            RAA_Matching_PL = abs(SGLI_RAA_PL - AHI_RAA)
    #                         print(RAA_Matching_PL)
        #                     
                            if RAA_Matching_PL < 10:
        #                         Load AHI/SGLI SZA
                                AHI_SZA_file_name = download_AHI(AHI_SZA_t , date, TIME, 'sza')
                                if AHI_SZA_file_name != None:


                                    Decompression(AHI_SZA_t,AHI_SZA_file_name)
                                    with open(AHI_SZA_t + AHI_SZA_file_name, 'rb') as fp:
                                        AHI_SZA = np.frombuffer(fp.read(),dtype='>f4').reshape(3000,3000)[urow:drow,lcol:rcol]
                                    AHI_SZA = np.mean(AHI_SZA)
                                    SGLI_SZA_PL = np.nanmean(SGLI_2_AHI_GEO(reporjection_file + SGLI_SZA_filename))

                                    SZA_Matching_PL = abs(math.cos(math.radians(SGLI_SZA_PL)) - math.cos(math.radians(AHI_SZA)))

                                    if SZA_Matching_PL < 0.02:


                                        MATCH_RESULT.append(date_new + TIME)
                                        MATCH_RESULT.append(date)
                                        print('PL')
                                        print('AHI:{}'.format(date_new + TIME))
                                        print('SGLI:{}'.format(date))
                                        print("AHI_RAA",AHI_RAA,"SGLI_RAA",SGLI_RAA_PL)
                                        print("AHI_VZA",AHI_VZA,"SGLI_VZA",SGLI_VZA_PL)
                                        print("AHI_SZA",AHI_SZA,"SGLI_SZA",SGLI_SZA_PL)





    
with open("{}_PL.txt".format(site_name),"w") as f:
    f.writelines('\n'.join(MATCH_RESULT))              

/home/liwei/.local/lib/python3.8/site-packages/skimage/measure/block.py:89: RuntimeWarning: Mean of empty slice
  return func(blocked, axis=tuple(range(image.ndim, blocked.ndim)),
/tmp/ipykernel_1203962/2797067998.py:25: RuntimeWarning: Mean of empty slice
  SGLI_VZA_PL = np.nanmean(SGLI_VZA_PL)
/home/liwei/.local/lib/python3.8/site-packages/skimage/measure/block.py:89: RuntimeWarning: Mean of empty slice
  return func(blocked, axis=tuple(range(image.ndim, blocked.ndim)),
/tmp/ipykernel_1203962/2797067998.py:25: RuntimeWarning: Mean of empty slice
  SGLI_VZA_PL = np.nanmean(SGLI_VZA_PL)
/home/liwei/.local/lib/python3.8/site-packages/skimage/measure/block.py:89: RuntimeWarning: Mean of empty slice
  return func(blocked, axis=tuple(range(image.ndim, blocked.ndim)),
/tmp/ipykernel_1203962/2797067998.py:25: RuntimeWarning: Mean of empty slice
  SGLI_VZA_PL = np.nanmean(SGLI_VZA_PL)
/home/liwei/.local/lib/python3.8/site-packages/skimage/measure/block.py:89: RuntimeWarning: Mean of empty sli

PL
AHI:201902050730
SGLI:20190205
AHI_RAA 97.42988203370842 SGLI_RAA 105.40127500000004
AHI_VZA 57.665504326690154 SGLI_VZA 58.99666875
AHI_SZA 50.94385 SGLI_SZA 51.52848749999999
PL
AHI:201902050740
SGLI:20190205
AHI_RAA 100.04136274659905 SGLI_RAA 105.40127500000004
AHI_VZA 57.665504326690154 SGLI_VZA 58.99666875
AHI_SZA 52.205086 SGLI_SZA 51.52848749999999


/home/liwei/.local/lib/python3.8/site-packages/skimage/measure/block.py:89: RuntimeWarning: Mean of empty slice
  return func(blocked, axis=tuple(range(image.ndim, blocked.ndim)),
/tmp/ipykernel_1203962/2797067998.py:25: RuntimeWarning: Mean of empty slice
  SGLI_VZA_PL = np.nanmean(SGLI_VZA_PL)
/home/liwei/.local/lib/python3.8/site-packages/skimage/measure/block.py:89: RuntimeWarning: Mean of empty slice
  return func(blocked, axis=tuple(range(image.ndim, blocked.ndim)),
/tmp/ipykernel_1203962/2797067998.py:25: RuntimeWarning: Mean of empty slice
  SGLI_VZA_PL = np.nanmean(SGLI_VZA_PL)
/home/liwei/.local/lib/python3.8/site-packages/skimage/measure/block.py:89: RuntimeWarning: Mean of empty slice
  return func(blocked, axis=tuple(range(image.ndim, blocked.ndim)),
/tmp/ipykernel_1203962/2797067998.py:25: RuntimeWarning: Mean of empty slice
  SGLI_VZA_PL = np.nanmean(SGLI_VZA_PL)
/home/liwei/.local/lib/python3.8/site-packages/skimage/measure/block.py:89: RuntimeWarning: Mean of empty sli

PL
AHI:201902200720
SGLI:20190220
AHI_RAA 98.06470869386467 SGLI_RAA 105.53694999999999
AHI_VZA 57.665504326690154 SGLI_VZA 57.6638
AHI_SZA 45.434883 SGLI_SZA 46.93333125
PL
AHI:201902200730
SGLI:20190220
AHI_RAA 100.96760176027092 SGLI_RAA 105.53694999999999
AHI_VZA 57.665504326690154 SGLI_VZA 57.6638
AHI_SZA 46.720543 SGLI_SZA 46.93333125
PL
AHI:201902200740
SGLI:20190220
AHI_RAA 103.73157881105217 SGLI_RAA 105.53694999999999
AHI_VZA 57.665504326690154 SGLI_VZA 57.6638
AHI_SZA 48.093315 SGLI_SZA 46.93333125


/home/liwei/.local/lib/python3.8/site-packages/skimage/measure/block.py:89: RuntimeWarning: Mean of empty slice
  return func(blocked, axis=tuple(range(image.ndim, blocked.ndim)),
/tmp/ipykernel_1203962/2797067998.py:25: RuntimeWarning: Mean of empty slice
  SGLI_VZA_PL = np.nanmean(SGLI_VZA_PL)
/home/liwei/.local/lib/python3.8/site-packages/skimage/measure/block.py:89: RuntimeWarning: Mean of empty slice
  return func(blocked, axis=tuple(range(image.ndim, blocked.ndim)),
/tmp/ipykernel_1203962/2797067998.py:25: RuntimeWarning: Mean of empty slice
  SGLI_VZA_PL = np.nanmean(SGLI_VZA_PL)
/home/liwei/.local/lib/python3.8/site-packages/skimage/measure/block.py:89: RuntimeWarning: Mean of empty slice
  return func(blocked, axis=tuple(range(image.ndim, blocked.ndim)),
/tmp/ipykernel_1203962/2797067998.py:25: RuntimeWarning: Mean of empty slice
  SGLI_VZA_PL = np.nanmean(SGLI_VZA_PL)


KeyboardInterrupt: 

In [6]:
AHI_SZA_t + AHI_SZA_file_name

'/media/liwei/Data/AHI_Angle/Solar_zenith_angle/201912231100.sun.zth.fld.4km.bin'